In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import logging

from kyle.sampling.fake_clf import DirichletFC, MultiDirichletFC
from kyle.evaluation import EvalStats, compute_accuracy, compute_ECE, compute_expected_max
from kyle.transformations import *
from kyle.calibration.calibration_methods import TemperatureScaling

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import pytorch_lightning as pl

import scipy.stats
import scipy.optimize
import numpy as np
import matplotlib.pyplot as plt

# Realistic Fake Classifiers

It's good to have a model of what a realistic Fake Classifier should look like.

Probably the simplest model for the fake classifier confidence vector distribution is the **Dirichlet Fake Classifier**:

\begin{equation}
C \sim Dirichlet(\alpha_1, \alpha_2, \alpha_3, ...)
\end{equation}

This classifier has a total of 'num_classes' parameters

However, this model is possibly a bit too simple as it only has a single local maximum in the distribution. A realistic fake classifier might for example have multiple local maxima in each of the corners of the simplex, i.e. it generally is very confident in its prediction and only very rarely uncertain (center of simplex). Something similar can actually be achieved using the Dirichlet distribution by setting all the parameters $\alpha_n < 1$. This pushes the distribution out into the corners, BUT however also out onto the sides of the simplex, which is not quite what we want. The center of a side of the simplex corresponds to a confidence vector $\vec\alpha = (1/\text{num_classes}-1, 1/\text{num_classes}-1, ..., 1/\text{num_classes}-1, 0)$, i.e. very uncertain in all but one of the classes.

Therefore we also consider two other Fake Classifiers that can have multiple local maxima, one in each of the corners, and therefore possibly represent real neural networks better:

Firstly the **Multi-Dirichlet Fake Classifier**:

\begin{align}
K & \sim Catgeorical(p_1, p_2, p_3, ...) \\
C & \sim Dirichlet_k(\sigma_k\cdot[1, 1, ..., 1, \alpha_k, 1, ...])
\end{align}

This classifier has a total of '3 x num_classes' parameters

i.e. we first draw from a K-categorical distribution and based on the result we then draw from one of K Dirichlet distributions. Each of the K Dirichlet distributions has two parameters $\sigma$ and $\alpha_k$ which represent the width and position of the local maximum in the k-th corner of the simplex.

Note: The pdf of this mixture distribution will be a weighted sum of the individual dirichlet distributions

Secondly the **Multi-Gaussian Fake Classifier**:

K-Categorical followed by one of K Gaussians followed by softmax

(probably doesn't make too much sense as Multi-Gaussian pdf is analytically intractable due to the softmax transformation)

**Note**:

When talking about the simple dirichlet FC $\vec{\alpha}$ refers to the vector of $\alpha$ parameters of the dirichlet distribution.

When talking about the Multi-Dirichlet FC $\vec{\alpha_k}$ refers to the vector of $\alpha_k$ parameters, and should not be confused to be a parameter vector of a single dirichlet.

In order to get an actually realistic Fake Classifier we use these three Fake Classifier models and fit their distributions to the observed confidence vector distributions for a couple of different neural networks.

In this case we use:

**LeNet 5** on CIFAR 10

**ResNet 20** on CIFAR 10

**ResNet 110** on CIFAR 10

In [ ]:
#load cifar dataset
#normalizarion also from https://github.com/akamaster/pytorch_resnet_cifar10
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

cifar_train_set = datasets.CIFAR10(os.getcwd(), train=True, download=True,
        transform=transforms.Compose([transforms.ToTensor(), normalize]))
cifar_test_set = datasets.CIFAR10(os.getcwd(), train=False, download=True,
                transform=transforms.Compose([transforms.ToTensor(), normalize]))

cifar_train = torch.utils.data.DataLoader(cifar_train_set, batch_size=4, shuffle=True, num_workers=2)
cifar_test = torch.utils.data.DataLoader(cifar_test_set, batch_size=4, shuffle=False, num_workers=2)

In [ ]:
#Small simple LeNet5 for CIFAR 10 classification

class lenet5(pl.LightningModule):

    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), 2)
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def training_step(self, batch, batch_idx):
        x, target = batch
        output = self(x)
        loss = F.cross_entropy(output, target)
        return loss
    
    def validation_step(self, batch, batch_idx):
        x, target = batch
        output = self(x)
        loss = F.cross_entropy(output, target)
        self.log('val_loss', loss, on_step=False, on_epoch=True, prog_bar=True)
            
    def configure_optimizers(self):
        optimizer = torch.optim.SGD(self.parameters(), lr=0.001, momentum=0.9)
        return optimizer
    

In [ ]:
#Proper implementation of ResNet18 for Cifar10. Pytorch only has ResNets for ImageNet which
#differ in number of parameters
#Code taken from: https://github.com/akamaster/pytorch_resnet_cifar10

def _weights_init(m):
    classname = m.__class__.__name__
    #print(classname)
    if isinstance(m, nn.Linear) or isinstance(m, nn.Conv2d):
        init.kaiming_normal_(m.weight)

class LambdaLayer(nn.Module):
    def __init__(self, lambd):
        super(LambdaLayer, self).__init__()
        self.lambd = lambd

    def forward(self, x):
        return self.lambd(x)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1, option='A'):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes:
            if option == 'A':
                """
                For CIFAR10 ResNet paper uses option A.
                """
                self.shortcut = LambdaLayer(lambda x:
                                            F.pad(x[:, :, ::2, ::2], (0, 0, 0, 0, planes//4, planes//4), "constant", 0))
            elif option == 'B':
                self.shortcut = nn.Sequential(
                     nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                     nn.BatchNorm2d(self.expansion * planes)
                )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 16

        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.layer1 = self._make_layer(block, 16, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 32, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 64, num_blocks[2], stride=2)
        self.linear = nn.Linear(64, num_classes)

        self.apply(_weights_init)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion

        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.avg_pool2d(out, out.size()[3])
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out
    

def resnet20():
    return ResNet(BasicBlock, [3, 3, 3])


def resnet32():
    return ResNet(BasicBlock, [5, 5, 5])


def resnet44():
    return ResNet(BasicBlock, [7, 7, 7])


def resnet56():
    return ResNet(BasicBlock, [9, 9, 9])


def resnet110():
    return ResNet(BasicBlock, [18, 18, 18])


def resnet1202():
    return ResNet(BasicBlock, [200, 200, 200])

In [ ]:
#Train a LeNet5
#Load selftrained LeNet 5 and pretrained Resnet20 and Resnet110 (don't have a dedicated GPU at hand D:)
#Pretrained nets taken from https://github.com/akamaster/pytorch_resnet_cifar10

#selftrained_lenet5 = lenet5()
#checkpoint_callback = pl.callbacks.ModelCheckpoint(monitor='val_loss', save_top_k=1, save_last=True)
#trainer = pl.Trainer(max_epochs=20, logger=False, checkpoint_callback=checkpoint_callback)
#trainer.fit(selftrained_lenet5, cifar_train, cifar_test)

selftrained_lenet5 = lenet5.load_from_checkpoint('./trained_models/lenet5.ckpt')

pretrained_resnet20 = resnet20()
pretrained_resnet110 = resnet110()

pretrained_resnet20_dict = torch.load('./trained_models/resnet20-12fca82f.th',
                               map_location=torch.device('cpu'))['state_dict']
pretrained_resnet20_dict = {key.replace("module.", ""): value for key, value in pretrained_resnet20_dict.items()}
pretrained_resnet20.load_state_dict(pretrained_resnet20_dict)

pretrained_resnet110_dict = torch.load('./trained_models/resnet110-1d1ed7c2.th',
                               map_location=torch.device('cpu'))['state_dict']
pretrained_resnet110_dict = {key.replace("module.", ""): value for key, value in pretrained_resnet110_dict.items()}
pretrained_resnet110.load_state_dict(pretrained_resnet110_dict)

**Set which neural net to fit here**

In [ ]:
#neural_net = selftrained_lenet5
neural_net = pretrained_resnet20
#neural_net = pretrained_resnet110

In [ ]:
#Get NN predictions on CIFAR10 test set

cifar_test_full = torch.utils.data.DataLoader(cifar_test_set, batch_size=len(cifar_test_set),
                                              shuffle=False, num_workers=2)
images, labels = next(iter(cifar_test_full))

neural_net.eval()

with torch.no_grad():
    
    logits = neural_net(images)
    prob = F.softmax(logits, dim=1)
    _, predicted = torch.max(prob, dim=1)
    print(f'NLL = {F.cross_entropy(logits, labels)}')
    print(f'accuracy = {(predicted == labels).sum().item() / labels.size(0)}')
    
gt_labels = labels.numpy()
confidences = prob.numpy()


In [ ]:
gt_labels_copy = gt_labels.copy()
confidences_copy = confidences.copy()

In [ ]:
gt_labels = gt_labels_copy.copy()
confidences = confidences_copy.copy()

In [ ]:
print(f'num non normalized confidence vectors = {np.sum((np.sum(confidences, axis=1) - 1) >= 1e-10)}')

#confidences are not perfectly normalized due to floating point error
#scipy.stats.dirichlet.pdf is very picky about normalization
#convert confidences to float64 first for better/more accurate normalization

confidences = np.array(confidences, dtype='float64')
confidences = confidences / np.sum(confidences, axis=1)[:,None]
print(f'num non normalized confidence vectors = {np.sum((np.sum(confidences, axis=1) - 1) >= 1e-10)}')


# Fitting Fake Classifiers using MLE

(Note: Any references to fitted results/distributions/graphs in the following sections used the LeNet5 as the 'real' neural network)

Having gotten the confidences of our neural net on the CIFAR 10 test set we can now try and fit an appropiate fake classifier to them. This can be done quite easily for the Dirichlet and Multi-Dirichlet FC's using MLE, as we have relatively simple expressions for the distribution pdf's. The Multi-Gaussian FC is not as easy as the softmax function complicates the fake classifier's pdf. (It would be necessary to invert the softmax function, which is only possible up to an additive constant. As a result the Multi-Gaussian fake classifier's pdf will be an integral over a gaussian mixture model's pdf.)

## 'Normal' MLE Fitting

MLE is probably the easiest and simplest approach when the fake classifier's pdf is known exactly. We calculate the negative log likelihood of our neural net's confidence vector distribution under the assumption of either a Dirichlet or Multi-Dirichlet distribution. Using one of scipy's many minimization algorithms/functions we can find the set of parameters of the fake classifier that maximize the log-likelihood/minimize the negative log-likelihood.

As always with fitting the choice of minimization algorithm, initial guesses and bounds is important:

As discussed at the start a somewhat alright fake classifier can possibly be achieved by using a simple Dirichlet Fake Classifier with alpha parameters $\alpha_n < 1$. For fitting the DirichletFC appropiate initial guesses and bounds might therefore be $\vec{\alpha}_\text{init} = (1,1,1,1,...)$  and $\alpha_\min, \alpha_\max = (0.0001, \text{None})$

As dicussed at the start the reasoning behind the Multi-Dirichlet FC is that each separate Dirichlet can be used to create a local maximum in one of the corners of the simplex. This only works if the full alpha vector of each dirichlet has all entries $>1$ (if any entry is $<1$ a local maximum does not exist), which means for each Dirichlet wee need $\alpha_k >1$ and $\sigma_k>1$. We also expect the maxima to be very 'squished' into the corners. i.e. $\alpha_k$ to be large. For fitting the Multi-Dirichlet FC appropate initial guesses and bounds might therefore be $\vec{\alpha}_{k,\text{init}} = (10,10,10,10,...)$ $\vec{\sigma}_\text{init} = (2,2,2,2,...)$ (squished into corners) and $\alpha_{k,\min}, \alpha_{k,\max} = (1, \text{None})$ $\sigma_\min, \sigma_\max = (1, \text{None})$

In [ ]:
#Save the 'EvalStats' all the fitted FC's for easier comparison later
#Very ugly but I didnt want to rename everything
save_fc_eval = []
save_NLL     = []

In [ ]:
#Fit Dirichlet FC to the test set confidence vector distributions using MLE fitting

num_classes = confidences.shape[1]

Dir_FC = DirichletFC(num_classes)
Dir_NLL = lambda parm: -np.sum(np.log( Dir_FC.pdf(confidences, parm) ))

#initial guesses and bounds for fitting simple dirichlet
init_alpha = 1*np.ones(num_classes)
bounds_alpha = [(0.0001, None)] * num_classes

Dirichlet_bestfit = scipy.optimize.minimize(Dir_NLL, init_alpha, bounds=bounds_alpha, options={'disp': True})
save_NLL.append(Dirichlet_bestfit.fun)
print(f'final NLL = {Dirichlet_bestfit.fun}')

In [ ]:
#Fit Multi-Dirichlet FC to the test set confidence vector distributions using MLE fitting

num_classes = confidences.shape[1]

MultiDir_FC = MultiDirichletFC(num_classes)
MultiDir_NLL = lambda parm: -np.sum(np.log( MultiDir_FC.pdf(confidences, *np.split(parm,3)) ))

#initial guesses and bounds for fitting multi-dirichlet
init_alpha = 10*np.ones(num_classes)
init_sigma = 2*np.ones(num_classes)
init_distribution_weights = np.ones(num_classes) / num_classes
bounds_alpha = [(1, None)] * num_classes
bounds_sigma = [(1, None)] * num_classes
bounds_distribution_weights = [(0, 1)] * num_classes

MultiDirichlet_bestfit = scipy.optimize.minimize(MultiDir_NLL,
                                                 np.concatenate((init_alpha, init_sigma, init_distribution_weights)),
                                                 bounds=bounds_alpha + bounds_sigma + bounds_distribution_weights,
                                                 options={'disp': True})
save_NLL.append(MultiDirichlet_bestfit.fun)
print(f'final NLL = {MultiDirichlet_bestfit.fun}')

In [ ]:
#Fitted FC parameters
print(f'alpha = {Dirichlet_bestfit.x}\n')
print('alpha_k = {}\n\nsigma_k = {}\n\ndistribution_weights = {}'.format(*np.split(MultiDirichlet_bestfit.x,3)))

In [ ]:
#Set FC parameters to those found from MLE fit
Dir_FC.set_alpha(Dirichlet_bestfit.x)
MultiDir_FC.set_parameters(*np.split(MultiDirichlet_bestfit.x,3))

In [ ]:
#visual comparison of neural network and FC distributions
Dir_FC_eval = EvalStats(*Dir_FC.get_sample_arrays(50000))
MultiDir_FC_eval = EvalStats(*MultiDir_FC.get_sample_arrays(50000))
NNet_eval = EvalStats(gt_labels, confidences)

Dir_FC_eval.plot_confidence_distributions([0, "top_class"])
MultiDir_FC_eval.plot_confidence_distributions([0,"top_class"])
NNet_eval.plot_confidence_distributions([0,"top_class"])

for i in range(3):

    Dir_FC_eval.plot_confidence_distributions([i])
    MultiDir_FC_eval.plot_confidence_distributions([i])
    NNet_eval.plot_confidence_distributions([i])

In [ ]:
save_fc_eval.append(Dir_FC_eval)
save_fc_eval.append(MultiDir_FC_eval)

Above we have plotted the top class confidence distribution as well as the marginal confidence distributions for the actual neural network and our fitted fake classifiers (the full 10D confidence vector distribution is unfortuanetly a teeny bit difficult to visualise). This allows us to visually inspect how well our fake classifiers really reflect the true confidence distribution of our classifier. The 1st, 4th, ... graphs are the simple dirichlet FC. The 2nd, 5th, ... graphs are the multi-dirichlet FC. The 3rd, 6th, ... graphs are the real neural network confidence distributions.

As expected the simple dirichlet FC does not work well. It doesn't capture the multimodal nature of the true marginal distribution nor the high frequency of p=1.0 confidences in the top class confidence distributions.

The Multi-dirichlet FC seems to be a bit better capturing both of these qualities. However the maxima of the distributions don't lie far enough at the extremes, which is likely due to the fact that the local maximum of a dirichlet only lies exactly on the corner in the limit of $\alpha_k \rightarrow \infty$

Also of note is that the Multi-Dirichlet FC always tries to fit $\sigma < 1$, always reaching the lower bound of $\sigma = 1$ which we have imposed.

## 'Sufficiently Confident/Class Split' MLE Fitting 

Given the slight difficulties with fitting a 30 parameter Multi-Dirichlet model we try to simplify the fitting a bit. This can be done by assuming a scenario very similar to that of sufficiently confident fake classifiers. We assume that if we draw a confidence vector from the k'th dirichlet distribution, then that confidence vector always predicts the class k. That means:

\begin{equation}
P(\text{argmax}(\vec{c}) = k | \vec{c} \sim \text{Dir}_k) \simeq 1 \text{  or equivalently  } P(\text{argmax}(\vec{c}) = k) \simeq p_k 
\end{equation}

where $p_k$ is the weight of the k'th dirichlet distribution. This is of course only a good approximation if $P(\text{argmax}(\vec{c}) \neq k | \vec{c} \sim \text{Dir}_k) \simeq 0$, i.e. if the individual Dirichlet distributions are sufficiently concentrated in their respective corner of the simplex with only negligible probability mass near the other corners of the simplex.

From the marginal class confidence distribution of the real neural network, we can see that this might be a very good approximation, as the class confidences always either lie near $c_k = 1.0$ or $c_k = 0.0$.

This approximation makes two simplifications possible:

1. We no longer have to fit the distribution weights $p_k$. Instead they can simply be read off from the real neural networks predicted class probabilities. This reduces the number of fitted parameters from 30 to 20

2. We can now fit each Dirichlet distribution from the Multi-Dirichlet Fake classifier individually to the subset of the data for which $\text{argmax}(\vec{c}) = k$. This reduces the problem further from a simultaneous fit of 20 parameters to 10 fits of 2 parameters each.


We proceed in the following way: First we split the real neural networks confidence distribution by predicted class into 10 subsets. Using the relative size of these subest we estimate the distribution weights of the dirichlet distributions in our Multi-Dirichlet FC. We then fit each dirichlet distribution of our Multi-Dirichlet FC separately to its corresponding data set, by minimizing the negative log likelihood as before.

Note that as we are now fitting each dirichlet distribution of the Multi-Dirichlet fake classifier individually we can no longer use MultiDirichletFC.pdf() directly, instead we have to use the pdf of a single dirichlet, DirichletFC.pdf(), with a parameter vector $\sigma_k\cdot[1, 1, ..., 1, \alpha_k, 1, ...]$ for our NLL.

In [ ]:
#split the full confidence vector distribution of the real nn into subsets by predicted class
num_classes = confidences.shape[1]
class_split_confidences = [confidences[np.argmax(confidences, axis=1)==i, :] for i in range(num_classes)]

In [ ]:
#estimate Multi-Dirichlet distribution weights from the real neural networks predicted class distribution
estimated_distribution_weights = [k_class_confidences.shape[0] for k_class_confidences in class_split_confidences]
estimated_distribution_weights = estimated_distribution_weights / np.sum(estimated_distribution_weights)

In [ ]:
Dir_FC = DirichletFC(num_classes)
MultiDir_FC = MultiDirichletFC(num_classes)

alpha_k = []
sigma_k = []
NLL_k   = []

for k, k_class_confidences in enumerate(class_split_confidences):
    
    def k_dir_NLL(alpha, sigma):
        alpha_vector = np.ones(num_classes)
        alpha_vector[k] = alpha
        alpha_vector *= sigma
        return -np.sum(np.log( Dir_FC.pdf(k_class_confidences, alpha_vector) ))

    #initial guesses and bounds for fitting individual dirichlets
    init_alpha = 10
    init_sigma = 2
    bounds_alpha = [(1, None)]
    bounds_sigma = [(1, None)]

    k_dir_bestfit = scipy.optimize.minimize(lambda parms: k_dir_NLL(*parms), np.array([init_alpha, init_sigma]),
                                            bounds = bounds_alpha + bounds_sigma,
                                            options={'disp': True})
    
    alpha_k.append(k_dir_bestfit.x[0])
    sigma_k.append(k_dir_bestfit.x[1])
    NLL_k.append(k_dir_bestfit.fun)
    
save_NLL.append(np.sum(NLL_k))
print(f'final NLL = {np.sum(NLL_k)}')

In [ ]:
#set Multi-Dirichlet FC parameters to estimated distribution weights and fitted alpha_k and sigma_k values
MultiDir_FC.set_parameters(alpha_k, sigma_k, estimated_distribution_weights)
print('alpha_k = {}\n\nsigma_k = {}\n\ndistribution_weights = {}'.format(*MultiDir_FC.get_parameters()))

Similar to above we can now visiually compare the marginal and top class confidence distributions of our fake classifier and the real neural network:

In [ ]:
MultiDir_FC_eval = EvalStats(*MultiDir_FC.get_sample_arrays(50000))
NNet_eval = EvalStats(gt_labels, confidences)

MultiDir_FC_eval.plot_confidence_distributions([0,"top_class"])
NNet_eval.plot_confidence_distributions([0,"top_class"])

for i in range(3):

    MultiDir_FC_eval.plot_confidence_distributions([i])
    NNet_eval.plot_confidence_distributions([i])

In [ ]:
save_fc_eval.append(MultiDir_FC_eval)

This doesn't really seem to work much better than fitting all 30 parameters at the same time (although it is definietly much faster in terms of computational time).

## Sigma < 1

Interestingly enough it runs into the same problem of always trying to fit very small $\sigma_k < 1$, even though the sensible/intuitve choice would be $\sigma_k > 1$. This indicates that maybe our sensible/intuitve choice of wanting $\sigma_k > 1$ isn't actually that sensible. As explained earlier the reasoning behind the Multi-Dirichlet model is that each Dirichlet distribution is used to create a local maximum in one of the corners of the simplex, with $\alpha_k$ determining how squished into the corner and $\sigma_k$ how broad the local maximum is. The requirement of $\sigma_k > 1$ and $\alpha_k > 1$ arose, because if any parameter of the Dirichlet is $< 1$ a local maximum does not exist and our whole reasoning behind introducing the Multi-Dirichlet fails.

However, as it turns out a reasonable parameter choice might also be $\sigma_k < 1$, with $\alpha_k \cdot \sigma_k > 1$. This would mean that all parameters of the dirichlet, with exception of the k'th entry, are $ < 1$. While a local maximum no longer exists (we now have divergences at the sides and corners) this can still lead to an equivalent situation with most of the probability mass concentrated in one corner of the simplex. Below is a quick and dirty visualisation of this:

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20,9))

vis_dir_FC = DirichletFC(3)

vis_dir_FC.set_alpha([0.2, 20, 0.2])
_, vis_confidences = vis_dir_FC.get_sample_arrays(10000)

ax[0].scatter(vis_confidences[:,0], vis_confidences[:,1], s=0.1)
ax[0].plot([0,1], [1,0], 'r--')
ax[0].plot([0,1], [0,0], 'r--')
ax[0].plot([0,0], [0,1], 'r--')
ax[0].set_xlim([-0.2,1.2])
ax[0].set_ylim([-0.2,1.2])

vis_dir_FC.set_alpha([2, 50, 2])
_, vis_confidences = vis_dir_FC.get_sample_arrays(10000)

ax[1].scatter(vis_confidences[:,0], vis_confidences[:,1], s=0.1)
ax[1].plot([0,1], [1,0], 'r--')
ax[1].plot([0,1], [0,0], 'r--')
ax[1].plot([0,0], [0,1], 'r--')
ax[1].set_xlim([-0.2,1.2])
ax[1].set_ylim([-0.2,1.2])

Below we have repeated both the direct Multi-Dirichlet FC fitting as well as the 'class split' fitting, but this time with a lower bound for sigma of $\sigma_\text{lower} = 0.001$. Allowing for this seems to improve things a bit, but it is still a far cry from properly resembling the true marginal and top class confidence distributions.

### First the simultaneous fit of 30 parameters:

In [ ]:
#Fit Multi-Dirichlet FC to the test set confidence vector distributions using MLE fitting

num_classes = confidences.shape[1]

MultiDir_FC = MultiDirichletFC(num_classes)
MultiDir_NLL = lambda parm: -np.sum(np.log( MultiDir_FC.pdf(confidences, *np.split(parm,3)) ))

#initial guesses and bounds for fitting multi-dirichlet
init_alpha = 10*np.ones(num_classes)
init_sigma = 2*np.ones(num_classes)
init_distribution_weights = np.ones(num_classes) / num_classes
bounds_alpha = [(1, None)] * num_classes
bounds_sigma = [(0.001, None)] * num_classes
bounds_distribution_weights = [(0, 1)] * num_classes

MultiDirichlet_bestfit = scipy.optimize.minimize(MultiDir_NLL,
                                                 np.concatenate((init_alpha, init_sigma, init_distribution_weights)),
                                                 bounds=bounds_alpha + bounds_sigma + bounds_distribution_weights,
                                                 options={'disp': True})

save_NLL.append(MultiDirichlet_bestfit.fun)
print(f'final NLL = {MultiDirichlet_bestfit.fun}')

In [ ]:
#Set FC parameters to those found from MLE fit
MultiDir_FC.set_parameters(*np.split(MultiDirichlet_bestfit.x,3))
print('alpha_k = {}\n\nsigma_k = {}\n\ndistribution_weights = {}'.format(*np.split(MultiDirichlet_bestfit.x,3)))

In [ ]:
#visual comparison of neural network and FC distributions
MultiDir_FC_eval = EvalStats(*MultiDir_FC.get_sample_arrays(50000))
NNet_eval = EvalStats(gt_labels, confidences)

MultiDir_FC_eval.plot_confidence_distributions([0,"top_class"])
NNet_eval.plot_confidence_distributions([0,"top_class"])

for i in range(3):

    MultiDir_FC_eval.plot_confidence_distributions([i])
    NNet_eval.plot_confidence_distributions([i])

In [ ]:
save_fc_eval.append(MultiDir_FC_eval)

### And secondly the 10 fits of 2 parameters each:

In [ ]:
#split the full confidence vector distribution of the real nn into subsets by predicted class
num_classes = confidences.shape[1]
class_split_confidences = [confidences[np.argmax(confidences, axis=1)==i, :] for i in range(num_classes)]

In [ ]:
#estimate Multi-Dirichlet distribution weights from the real neural networks predicted class distribution
estimated_distribution_weights = [k_class_confidences.shape[0] for k_class_confidences in class_split_confidences]
estimated_distribution_weights = estimated_distribution_weights / np.sum(estimated_distribution_weights)

In [ ]:
Dir_FC = DirichletFC(num_classes)
MultiDir_FC = MultiDirichletFC(num_classes)

alpha_k = []
sigma_k = []
NLL_k   = []

for k, k_class_confidences in enumerate(class_split_confidences):
    
    def k_dir_NLL(alpha, sigma):
        alpha_vector = np.ones(num_classes)
        alpha_vector[k] = alpha
        alpha_vector *= sigma
        return -np.sum(np.log( Dir_FC.pdf(k_class_confidences, alpha_vector) ))

    #initial guesses and bounds for fitting individual dirichlets
    init_alpha = 10
    init_sigma = 2
    bounds_alpha = [(1, None)]
    bounds_sigma = [(0.001, None)]

    k_dir_bestfit = scipy.optimize.minimize(lambda parms: k_dir_NLL(*parms), np.array([init_alpha, init_sigma]),
                                            bounds = bounds_alpha + bounds_sigma,
                                            options={'disp': True})
    
    alpha_k.append(k_dir_bestfit.x[0])
    sigma_k.append(k_dir_bestfit.x[1])
    NLL_k.append(k_dir_bestfit.fun)
    
save_NLL.append(np.sum(NLL_k))
print(f'final NLL = {np.sum(NLL_k)}')

In [ ]:
#set Multi-Dirichlet FC parameters to estimated distribution weights and fitted alpha_k and sigma_k values
MultiDir_FC.set_parameters(alpha_k, sigma_k, estimated_distribution_weights)
print('alpha_k = {}\n\nsigma_k = {}\n\ndistribution_weights = {}'.format(*MultiDir_FC.get_parameters()))

In [ ]:
MultiDir_FC_eval = EvalStats(*MultiDir_FC.get_sample_arrays(50000))
NNet_eval = EvalStats(gt_labels, confidences)

MultiDir_FC_eval.plot_confidence_distributions([0,"top_class"])
NNet_eval.plot_confidence_distributions([0,"top_class"])

for i in range(3):

    MultiDir_FC_eval.plot_confidence_distributions([i])
    NNet_eval.plot_confidence_distributions([i])

In [ ]:
save_fc_eval.append(MultiDir_FC_eval)

## Further Modifications to Fitting Procedure

Other 'improvements' that can be made is to modify the fitting in two ways:

1. Scale/Transform the confidences that we are trying to fit

2. Scale/Transform the parameters we are trying to fit

See https://epub.wu.ac.at/4077/1/Report125.pdf for more info on fitting dirichlets

1:

An important problem that we haven't quite adressed/just ignored are divergences in the distribution and the log-likelihood. First of all an 'obvious' divergence occurs if any of the dirichlet parameters are $< 1$. For such a case the pdf on the sides/corners of the simplex corresponding to this parameter will have a divergence $\text{pdf} \rightarrow \infty$. As a result the log-likelihood will also diverge. Slightly less obvious however is that this divergence still occurs if the parameters of the dirichlet are $> 1$. In this case the pdf on the sides/corners of the simplex will be $= 0$, which also leads to a divergence in the log-lieklihood which we are trying to fit.

In short if any component of the confidence vector $= 0$ we run into divergences (and the confidence distribution of our real neural networks have a lot of $0$ confidence values)

To mitigate this we can rescale the confidences before fitting according to

\begin{equation}
\vec{c} = \frac{\vec{c}(N-1) + 1/\text{num_classes}}{N}
\end{equation}

where N is the number of samples.


2:

In general, introducing bounds into minimzation problems can complicate things. However instead of fitting the dirichlet parameters directly we can first rescale/transform them onto the interval $[-\infty, \infty]$ (Very similar to the idea of generalized linear models, just that we do not have predictor variables in our case).

For the simple dirichlet FC and appropiate rescaling would be $log(\cdot)$ mapping the allowed alpha interval $[0, \infty]$ onto $[-\infty, \infty]$

For the multi-dirichlet FC, with $\sigma > 0$ and $\alpha > 1$ an appropiate transform would be e.g. $log(\sigma)$ and $log(\alpha - 1)$

**Note**:

As it turns out rescaling values might not be that great of an idea. The problem is that the minimization algorithms end up taking stepsizes of up to $10$ in $\log(\alpha)$ which results in $\alpha \sim 10^{10}$. Even after rescaling the confidences to avoid 0 values, we still run into problems with such a large alpha. For a value of $c \sim 10^{-5}$, we get $c^\alpha \sim 10^{-50}$ which might as well be 0 and will lead to errors in the log-likelihood. Although this doesn't actually seem to affect results too much, it still finds the same minimum as a minimization without log transforms but with bounds

Let's try fitting the Multi-Dirichlet FC again in both ways, all 30 parameters directly and 10 times 2 parameters, while applying the two imporvements mentioned above

In [ ]:
#rescale the confidences to avoid 0 confidences and renormalize

num_classes = confidences.shape[1]

rescaled_confidences = (confidences * (confidences.shape[0] - 1) + 1/num_classes) / confidences.shape[0]
rescaled_confidences = rescaled_confidences / np.sum(rescaled_confidences, axis=1)[:,None]

### First the simultaneous fit of 30 parameters:

In [ ]:
MultiDir_FC = MultiDirichletFC(num_classes)

def MultiDir_NLL(log_alpha, log_sigma, logit_distribution_weights):
    alpha = np.exp(log_alpha)+1
    sigma = np.exp(log_sigma)
    distribution_weights = np.exp(logit_distribution_weights) / (1 + np.exp(logit_distribution_weights))
    return -np.sum(np.log( MultiDir_FC.pdf(rescaled_confidences, alpha, sigma, distribution_weights) ))
    
#initial guesses for fitting multi-dirichlet (no longer need bounds due to log transformations)
init_log_alpha = np.zeros(num_classes)
init_log_sigma = np.zeros(num_classes)
init_logit_distribution_weights = np.zeros(num_classes)

MultiDirichlet_bestfit = scipy.optimize.minimize(lambda parms: MultiDir_NLL(*np.split(parms,3)),
                                                 np.concatenate((init_log_alpha, init_log_sigma, init_logit_distribution_weights)),
                                                 options={'disp': True})
save_NLL.append(MultiDirichlet_bestfit.fun)
print(f'final NLL = {MultiDirichlet_bestfit.fun}')

In [ ]:
#Set FC parameters to those found from MLE fit
l_a, l_s, l_w = np.split(MultiDirichlet_bestfit.x,3)
MultiDir_FC.set_parameters(np.exp(l_a)+1, np.exp(l_s), np.exp(l_w) / (1 + np.exp(l_w)))

In [ ]:
print('alpha_k = {}\n\nsigma_k = {}\n\ndistribution_weights = {}'.format(*np.split(MultiDirichlet_bestfit.x,3)))

In [ ]:
MultiDir_FC_eval = EvalStats(*MultiDir_FC.get_sample_arrays(50000))
NNet_eval = EvalStats(gt_labels, confidences)

MultiDir_FC_eval.plot_confidence_distributions([0,"top_class"])
NNet_eval.plot_confidence_distributions([0,"top_class"])

for i in range(3):
    MultiDir_FC_eval.plot_confidence_distributions([i])
    NNet_eval.plot_confidence_distributions([i])

In [ ]:
save_fc_eval.append(MultiDir_FC_eval)

### And secondly the 10 fits of 2 parameters each:

In [ ]:
class_split_confidences = [rescaled_confidences[np.argmax(rescaled_confidences, axis=1)==i, :] for i in range(num_classes)]

estimated_distribution_weights = [k_class_confidences.shape[0] for k_class_confidences in class_split_confidences]
estimated_distribution_weights = estimated_distribution_weights / np.sum(estimated_distribution_weights)

In [ ]:
Dir_FC = DirichletFC(num_classes)
MultiDir_FC = MultiDirichletFC(num_classes)

alpha_k = []
sigma_k = []
NLL_k

for k, k_class_confidences in enumerate(class_split_confidences):
    
    def k_dir_NLL(log_alpha, log_sigma):
        alpha_vector = np.ones(num_classes)
        alpha_vector[k] = np.exp(log_alpha)+1 
        alpha_vector *= np.exp(log_sigma)
        return -np.sum(np.log( Dir_FC.pdf(k_class_confidences, alpha_vector) ))

    #initial guesses fitting individual dirichlets (no longer need bounds due to log transformations)
    init_log_alpha = 0
    init_log_sigma = 0

    k_dir_bestfit = scipy.optimize.minimize(lambda parms: k_dir_NLL(*parms), np.array([init_log_alpha, init_log_sigma]),
                                            options={'disp': True})
    
    alpha_k.append(np.exp(k_dir_bestfit.x[0])+1)
    sigma_k.append(np.exp(k_dir_bestfit.x[1]))
    NLL_k.append(k_dir_bestfit.fun)

save_NLL.append(np.sum(NLL_k))
print(f'final NLL = {np.sum(NLL_k)}')

In [ ]:
MultiDir_FC.set_parameters(alpha_k, sigma_k, estimated_distribution_weights)
print('alpha_k = {}\n\nsigma_k = {}\n\ndistribution_weights = {}'.format(*MultiDir_FC.get_parameters()))

In [ ]:
MultiDir_FC_eval = EvalStats(*MultiDir_FC.get_sample_arrays(50000))
NNet_eval = EvalStats(gt_labels, confidences)

MultiDir_FC_eval.plot_confidence_distributions([0,"top_class"])
NNet_eval.plot_confidence_distributions([0,"top_class"])

for i in range(3):
    MultiDir_FC_eval.plot_confidence_distributions([i])
    NNet_eval.plot_confidence_distributions([i])


In [ ]:
save_fc_eval.append(MultiDir_FC_eval)

For easier comparison the results are aggregated here

### Real NN's Confidence Distribution:

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(18,7))
plt.axes(axs[0])
NNet_eval.plot_confidence_distributions(["top_class"], new_fig = False)
plt.axes(axs[1])
NNet_eval.plot_confidence_distributions([0], new_fig = False)


### Dirichlet FC Confidence Distribution:

In [ ]:
print(f'final NLL = {save_NLL[0]}')
fig, axs = plt.subplots(1,2, figsize=(18,7))
plt.axes(axs[0])
save_fc_eval[0].plot_confidence_distributions(["top_class"], new_fig = False)
plt.axes(axs[1])
save_fc_eval[0].plot_confidence_distributions([0], new_fig = False)

### Directly Fitted Multi Dirichlet FC Confidence Distribution:

In [ ]:
print(f'final NLL = {save_NLL[1]}')
fig, axs = plt.subplots(1,2, figsize=(18,7))
plt.axes(axs[0])
save_fc_eval[1].plot_confidence_distributions(["top_class"], new_fig = False)
plt.axes(axs[1])
save_fc_eval[1].plot_confidence_distributions([0], new_fig = False)

### Class Split Fitted Multi Dirichlet FC Confidence Distribution:

In [ ]:
print(f'final NLL = {save_NLL[2]}')
fig, axs = plt.subplots(1,2, figsize=(18,7))
plt.axes(axs[0])
save_fc_eval[2].plot_confidence_distributions(["top_class"], new_fig = False)
plt.axes(axs[1])
save_fc_eval[2].plot_confidence_distributions([0], new_fig = False)

### Directly Fitted Multi Dirichlet FC Confidence Distribution, sigma < 1:

In [ ]:
print(f'final NLL = {save_NLL[3]}')
fig, axs = plt.subplots(1,2, figsize=(18,7))
plt.axes(axs[0])
save_fc_eval[3].plot_confidence_distributions(["top_class"], new_fig = False)
plt.axes(axs[1])
save_fc_eval[3].plot_confidence_distributions([0], new_fig = False)

### Class Split Fitted Multi Dirichlet FC Confidence Distribution, sigma < 1:

In [ ]:
print(f'final NLL = {save_NLL[4]}')
fig, axs = plt.subplots(1,2, figsize=(18,7))
plt.axes(axs[0])
save_fc_eval[4].plot_confidence_distributions(["top_class"], new_fig = False)
plt.axes(axs[1])
save_fc_eval[4].plot_confidence_distributions([0], new_fig = False)

### 'Improved' Directly Fitted Multi Dirichlet FC Confidence Distribution, sigma < 1:

In [ ]:
print(f'final NLL = {save_NLL[5]}')
fig, axs = plt.subplots(1,2, figsize=(18,7))
plt.axes(axs[0])
save_fc_eval[5].plot_confidence_distributions(["top_class"], new_fig = False)
plt.axes(axs[1])
save_fc_eval[5].plot_confidence_distributions([0], new_fig = False)

### 'Improved' Class Split Fitted Multi Dirichlet FC Confidence Distribution, sigma < 1:

In [ ]:
print(f'final NLL = {save_NLL[6]}')
fig, axs = plt.subplots(1,2, figsize=(18,7))
plt.axes(axs[0])
save_fc_eval[6].plot_confidence_distributions(["top_class"], new_fig = False)
plt.axes(axs[1])
save_fc_eval[6].plot_confidence_distributions([0], new_fig = False)

## Conclusion:

Comparing how well different models fit is always a bit of a tricky thing to do. The Likelihood ratio test only works for comparing nested models, i.e. models where one is a special/restricted case of the other model, which is unfortunately not the case here.

However, non-nested models can be compared (informally) using the Akaike Information Criterion (AIC) defines as:

\begin{equation}
AIC = 2k - 2\ln(\hat{L})
\end{equation}

where $k$ is the number of parameters of the model and $\hat{L}$ the maximum lilelihood.

Given that our models have parameters of either on the order of $\sim 10^1$ whereas $\ln(\hat{L}) \sim 10^6$ we can pretty much just compare likelihoods directly.

### For the LeNet 5 we conclude:

1. Suprisingly, the simple Dirichlet fits the real NN better than the Multi-Dirichlet with $\sigma > 1$, even though visually it looks like the Multi-Dirichlet captures/reproduces the marginal distributions of the real NN a lot better

2. Also suprisingly, fitting the Multi-Dirichlet directly gives worse results according to the AIC comared to fitting it via the approximation of 'class splitting' (even though, again, the directly fitted Multi-Dirichlet marginal distributions looks more 'real')

3. Allowing for $\sigma < 1$ improves the AIC a lot for both directly and class split fitted Multi-Dirichlet FC's

4. The 'Improvements' to the fitting, transforming data and parameters, seem to have varying effects: They improve class split fitting but make direct fitting worse and in general the log transforms seem to introduce more warnings and complications to the fitting than they get rid of.

The most promising results seem to therefore be achieved using the directly fitted Multi-Dirichlet FC with sigma < 1. While it does actually result in a slightly worse AIC than the same class-split FC the marginal distributions look a lot more 'real'.

### For the ResNet 20 we conclude:

Similar results as for the LeNet 5 with one major exception:

The difference between the class split and directly fitted multi dirichlet FC is negligibly small. This not a suprise as we expect the more complex NN's to be more confident in general, i.e. for the confidences to be more concentrated in the corners. This makes the more complex NN's closer to a 'sufficiently confident like' behaviour that we based the class split fitting approximation on.

Again the most promising results seem to be achieved using the directly fitted Multi-Dirichlet FC with sigma < 1.

### For the ResNet 110 we conclude:

Pretty much almost the same as for the ResNet 20

### In General:

The Multi-Dirichlet FC seems to be a lot better at reproducing the ResNet 20 and ResNet 110 confidence distributions. The LeNet 5 is more difficult for the Multi-Dirichlet FC to reproduce. This is likely due to the LeNet 5 not being 'overconfident enough', i.e. when looking at  the LeNet's 'top class' confidence distribution a fair amount of confidences lie in the middle of the distribution at $p \sim 0.5$. For ResNet 20 and ResNet 110 this is not the case, pretty much all confidences lie at the extreme value of $p \sim 1.0$.

The transform of the confidences to avoid 0 confidences is probably a good thing to keep.


Maybe TODO: Try other fitting methods: e.g. moment matching or fitting only to the marginal distributions

Maybe TODO: Try 'stochastic fitting' for the multi-gaussian FC where we sample the FC calculate the corrseponding marginal distributions and minimize e.g. the squared error loss between the neural network marginal distributions and the marginal distributions sampled from the FC. Could get around problem of multi-gaussian FC having analytically intractable pdf

TODO: Find a good simplex automorphism to also recreate the reliability diagrams of the neural networks.